# Example CIP API Usage

## The below functionality will allow you to authenticate in the CIP API with your credentials and programatically access the endpoints that are available to you

### This will load in the credentials you have exported in your shell (please see the README.md) or you can enter them here

In [8]:
import os
CIP_API_USERNAME = os.getenv("CIP_API_USERNAME") or "your_username_here"
CIP_API_PASSWORD = os.getenv("CIP_API_PASSWORD") or "your_password_here"
CIP_API_SERVER_URL = os.getenv("CIP_API_SERVER_URL") or "https://cip-api-server-url/api/{endpoint}"

### `get_url_json_response` below will return the json response from the url you provide (if successful) and will authenticate based on your credentials.

In [17]:
import requests


def get_authenticated_header():
    url = CIP_API_SERVER_URL
    auth_endpoint = "get-token/"

    irl_response = requests.post(
        url=url.format(endpoint=auth_endpoint),
        json=dict(
            username=CIP_API_USERNAME,
            password=CIP_API_PASSWORD,
        ),
    )
    irl_response_json = irl_response.json()
    token = irl_response_json.get('token')

    auth_header = {
        'Accept': 'application/json',
        "Authorization": "JWT {token}".format(token=token),
    }
    return auth_header

def get_url_json_response(url):
    response = requests.get(url=url, headers=get_authenticated_header())

    if response.status_code != 200:
        raise ValueError(
            "Received status: {status} for url: {url} with response: {response}".format(
            status=response.status_code, url=url, response=response.content)
        )
    return response.json()

### Create a string for the endpoint you wish to connect to
### A list of available endpoints can be viewed at https://cip-api-server-url/api/docs (replacing cip-api-server-url with the server you wish to connect to)

In [20]:
interpretation_request_list_endpoint = "interpretationRequestsList"

### Use `format` to construct the full URL

In [21]:
IR_list_url = CIP_API_SERVER_URL.format(endpoint=interpretation_request_list_endpoint)

### Pass this URL to `get_url_json_response` to get the json of all CIP API Interpretation Requests your credentials will allow you to view

In [22]:
IR_list = get_url_json_response(url=IR_list_url)

### Inspect the results

In [23]:
type(IR_list)

dict

In [24]:
IR_list.keys()

[u'count', u'previous', u'results', u'next']

In [25]:
type(IR_list['results'])

list

In [26]:
len(IR_list['results'])

100

In [28]:
IR_list['results'][0]

{u'case_priority': u'low',
 u'cip': u'omicia',
 u'cohort_id': u'50003384',
 u'family_id': u'50003384',
 u'files': [],
 u'interpretation_request_id': u'32-2',
 u'interpreted_genomes': [{u'cip_version': 1,
   u'clinical_reports': [],
   u'create_at': u'2017-01-16T14:12:22.773815Z'}],
 u'last_status': u'report_sent',
 u'last_update': u'2017-05-08 13:43:17',
 u'number_of_samples': 3,
 u'proband': u'50003384',
 u'sample_type': u'raredisease',
 u'sites': [u'GSTT'],
 u'tags': []}